In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

In [ ]:
def count(X):
    return X.shape[0]

def npdf(x, mean, var):
    return stats.norm.pdf(x, mean, var)

In [ ]:
data = []
with open("Datasets/DATA.txt","r") as f:
    data = np.array([l.split("  ") for l in f.read().splitlines()], dtype=float)

df = pd.DataFrame(
    data,
    columns = ["gender","age","risk factor","sbp","hr1","st","hr2","crt","kil","event"],
)

In [ ]:
df['gender'] = df['gender'].astype(int)
df['age'] = df['age'].astype(int)
df['risk factor'] = df['risk factor'].astype(int)
df['st'] = df['st'].astype(int)
df['kil'] = df['kil'].astype(int)
df['event'] = df['event'].astype(int)

In [ ]:
df.head()

In [ ]:
df.describe()

### Continuous Variables

* Age
* SBP
* HR Fused (HR1 + HR2)
* CRT

In [ ]:
var = np.array([2, 0.5])**2
gamma = np.sum(1 / var)
s = 1 / gamma
hr_fused = df.hr1/(gamma*var[0]) + df.hr2/(gamma*var[1])
df['hr_fused'] = hr_fused

In [ ]:
plt.figure(figsize=(10,5))
x = np.linspace(30,130, 100)
plt.plot(x, npdf(x, df.hr1.mean(), df.hr1.std()))
plt.plot(x, npdf(x, df.hr2.mean(), df.hr2.std()),'r')
plt.plot(x, npdf(x, df.hr_fused.mean(), df.hr_fused.std()),'g')
plt.legend(['HR BP','HR ECG','HR Fused'])

Testing for normality

In [ ]:
fig = plt.figure(figsize=(8,5))

plt.subplot(221)
plt.hist(df.age)
plt.title('Age')

plt.subplot(222)
plt.hist(df.sbp)
plt.title('Systolic Blood Pressure')

plt.subplot(223)
plt.hist(df.hr_fused)
plt.title('Heart Rate (Fused)')

plt.subplot(224)
plt.hist(df.crt)
plt.title('Creatinine')

fig.tight_layout()

In [ ]:
from scipy.stats import ks_1samp, shapiro
from scipy.stats import norm


p_values = {}
for c in ["age","sbp","hr_fused","crt"]:
    _, ks = ks_1samp(df[c], norm.cdf)
    _, s = shapiro(df[c])
    p_values[c] = {
        "ks": ks >= 0.05,
        "shapiro": s >= 0.05,
    }

In [ ]:
pd.DataFrame(p_values).T

It will be assumed that, given the results from the statistical tests and the histograms, only systolic blood presure (SBP) follows a normal distribution. As such, the remaining continuous variables will be discretized.

In [ ]:
fig = plt.figure(figsize=(10,3))
plot_bins = 7

plt.subplot(131)
bins_age = plt.hist(df.age, plot_bins)[1]
plt.title('Age')

plt.subplot(132)
bins_hr_fused = plt.hist(df.hr_fused, plot_bins)[1]
plt.title('Heart Rate (Fused)')

plt.subplot(133)
bins_crt = plt.hist(df.crt, plot_bins)[1]
plt.title('Creatinine')

fig.tight_layout()

In [ ]:
def discretize(df, bins, columns=["age","hr_fused","crt"]):
    '''
    Discretize continuous variables
    '''

    probs = {} 

    for c in columns:
        probs[c] = np.zeros((2,len(bins[c])-1))
        for i, b in enumerate(bins[c]):
            
            if i == len(bins[c]) - 1:
                break

            range = df[(df[c] >= bins[c][i]) & (df[c] < bins[c][i+1])]

            range_no_event = count(range[range.event == 0])
            range_event = count(range[range.event == 1])

            probs[c][:,i] = [range_no_event/(range_no_event + range_event), range_event/(range_no_event + range_event)]

    return probs

def binize(x, bins, columns=["age","hr_fused","crt"]):

    bin = {

    }

    for c in columns:
        for i in range(len(bins[c])-1):
            if (x[c] >= bins[c][i]) & (x[c] < bins[c][i+1]):
                bin[c] = i
                break
            elif (i == (len(bins[c])-2)):
                bin[c] = i

    return bin

In [ ]:
bins = {
        "age": bins_age,
        "hr_fused": bins_hr_fused,
        "crt": bins_crt
}
discretized_vars = discretize(df, bins)

In [ ]:
def calc(df, columns=["age","sbp","hr_fused","crt"]):
    '''
    Calculate mean and variance for every variable in columns for the case of event and no event
    '''

    mean_no_event = np.zeros(4)
    mean_event = np.zeros(4)
    std_no_event = np.zeros(4)
    std_event = np.zeros(4)
    for i,c in enumerate(columns):
        mean_no_event[i] = df[df.event == 0][c].mean()
        mean_event[i] = df[df.event == 1][c].mean()
        
        std_no_event[i] = df[df.event == 0][c].std()
        std_event[i] = df[df.event == 1][c].std()

    return {
        0: mean_no_event,
        1: mean_event,
    }, {
        0: std_no_event,
        1: std_event
    }

def bayes_countinuous(df, x, mean, std, columns=["age","sbp","hr_fused","crt"]):

    dist_no_event = []
    dist_event = []
    for i, c in enumerate(columns):
        dist_no_event.append(npdf(x[c], mean[0][i], std[0][i]))
        dist_event.append(npdf(x[c], mean[1][i], std[1][i]))

    return {
        0: np.prod(dist_no_event),
        1: np.prod(dist_event)
    }

In [ ]:
prob_no_event = count(df[df.event == 0])/count(df)
prob_event = count(df[df.event == 1])/count(df)

mu, var = calc(df,columns=["sbp"])

continuous_val = np.zeros((count(df), 2))
continuous_guess = np.zeros(count(df))

for i in range(count(df)):
    res = bayes_countinuous(df, df.iloc[i], mu, var, columns=["sbp"])
    
    continuous_val[i][0] = res[0] * prob_no_event
    continuous_val[i][1] = res[1] * prob_event

    if  continuous_val[i][1] > continuous_val[i][0] :
        continuous_guess[i] = 1
    else:
        continuous_guess[i] = 0
        

df['continuous_guess'] = continuous_guess.astype(int)
df.head()

### Discrete Variables

X = { Gender, RF, ST, KIL, risk* }

discretized = { Age, Heart Rate Fused, Creatinine}

In [ ]:
'''
IF CTR >= 1.3 AND ST ==  ---> RISK = 1
IF KIL >= 2              ---> RISK = 1
'''

df['risk'] = np.where(((df.crt >= 1.4) & (df.st == 1)) | (df.kil >= 2), 1, 0)

In [ ]:
def compute_likelihood(data, x, bins, columns = ['gender','risk factor','st','kil','risk'], discretized_columns = ["age","hr_fused","crt"]):

    prob_cond_no_event = []
    prob_cond_event = []
    for c in columns:
        prob_cond_no_event.append(count(data[(data.event == 0) & (data[c] == x[c])])/count(data[data.event == 0]))
        prob_cond_event.append(count(data[(data.event == 1) & (data[c] == x[c])])/count(data[data.event == 1]))

    x_bins = binize(df.iloc[i], bins)

    for c in discretized_columns:
        prob_cond_no_event.append(discretized_vars[c][0][x_bins[c]])
        prob_cond_event.append(discretized_vars[c][1][x_bins[c]])

    return [np.prod(prob_cond_no_event),np.prod(prob_cond_event)]

In [ ]:
prob_no_event = count(df[df.event == 0])/count(df)
prob_event = count(df[df.event == 1])/count(df)

discrete_val = np.zeros((count(df),2))
discrete_guess = np.zeros(count(df))
for i in range(count(df)):
    res = compute_likelihood(df, df.iloc[i], bins)
    
    discrete_val[i][0] = res[0] * prob_no_event
    discrete_val[i][1] = res[1] * prob_event

    if  discrete_val[i][1] > discrete_val[i][0] :
        discrete_guess[i] = 1
    else:
        discrete_guess[i] = 0
        

df['discrete_guess'] = discrete_guess.astype(int)
df.head()

### Evaluation

In [ ]:
df.head()

In [ ]:
decision = continuous_val * discrete_val
df['decision'] = (decision[:,1] > decision[:, 0]).astype(int)
df.head()

Let's measure performance in terms of specificity, sensitivity and f1-score

In [ ]:
def sensitivity(tp, fn):
    return tp/(tp+fn)
def specificity(fp, tn):
    return tn/(fp+tn)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

tn, fp, fn, tp = confusion_matrix(df.risk, df.decision).ravel()

print(f'Sensitivity: {sensitivity(tp, fn)}')
print(f'Specificity: {specificity(fp, tn)}')
print(f'F1 Score: {f1_score(df.risk, df.decision)}')